In [46]:
import requests
import re
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [95]:
import pandas as pd
import numpy as np

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [4]:
page = 1

In [5]:
url = 'http://grls.rosminzdrav.ru/Ree_orgCI2.aspx?numDoc=&Name_Org=&region=2000&adr=&OrgOut=2&order=num_doc&orderType=Asc&all=0&datedoc=&CiAcrOrg=0&status=1&except=0&isOld=0&date_doc_old=&num_doc_old=&name_org_old=&adr_old=&pageNum={}'

In [74]:
url_hosp = 'http://grls.rosminzdrav.ru/'

In [124]:
import tqdm

In [128]:
from time import sleep

10

In [130]:
df_list = []
bad_links = []
for page_id in tqdm.tqdm(range(1,76)):
    page = requests.get(url.format(page_id))
    page = page.content
    page = BeautifulSoup(page, 'lxml')
    table = page.findAll('table', attrs = {"id":"ctl00_plate_gr"})[0]
    table_df = pd.read_html(str(table))[0]
#     table_df['types'] = np.nan
    table_df.columns = table_df.ix[0]
    table_df = table_df.drop(0).reset_index()
    del table_df['index']
    links = []
    for i, tr in enumerate(table):
        if '<th' in str(tr) or isinstance(tr, bs4.element.NavigableString):
            continue
        else:
            for td in tr:
                if not (isinstance(td, bs4.element.NavigableString)):
                    re_links = re.findall(r'window.location.href=\'(.+?)\'',str(td))
                    if re_links:
                        links += [re_links[0]]
                        break
    for i, link in enumerate(links):
        try:
            link_hosp = url_hosp + link
            hosp_page = requests.get(link_hosp)
            hosp_page = hosp_page.content
            hosp_page = BeautifulSoup(hosp_page, 'lxml')
            table_profiles = hosp_page.findAll('table', attrs = {"id":"ctl00_plate_grdDetailShort"})
            if table_profiles:
                table_profiles = table_profiles[0]
                types_list = list(pd.read_html(str(table_profiles))[0][0])[1:]
                table_df.set_value(i, 'types', ';'.join(types_list))
            else:
                bad_links += [link_hosp]
        except Exception as e:
            print(e)
            sleep(10)
            continue
    df_list += [table_df]
    


 12%|██████▎                                              | 9/75 [01:46<13:04, 11.88s/it]

HTTPConnectionPool(host='grls.rosminzdrav.ru', port=80): Max retries exceeded with url: /CIAcrOrgTrial.aspx?ID=819C178A-F6F2-4EC4-97FC-ED4365569E4A (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x00000212BF9942E8>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))


 37%|███████████████████▍                                | 28/75 [07:28<13:39, 17.45s/it]

HTTPConnectionPool(host='grls.rosminzdrav.ru', port=80): Max retries exceeded with url: /CIAcrOrgTrial.aspx?ID=84892F1F-99F2-407F-B2BD-346133F7EF2D (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x00000212C0093C88>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))


 95%|█████████████████████████████████████████████████▏  | 71/75 [17:31<00:51, 12.80s/it]

HTTPConnectionPool(host='grls.rosminzdrav.ru', port=80): Max retries exceeded with url: /CIAcrOrgTrial.aspx?ID=FA7D5F36-1819-4C1A-ABC6-A7040DC4CEE2 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x00000212C04BA668>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))


100%|████████████████████████████████████████████████████| 75/75 [18:52<00:00, 15.01s/it]


In [132]:
len(df_list)

75

In [170]:
hosp_df = pd.concat(df_list, ignore_index=True)

In [171]:
hosp_df.to_csv('hosp_data.csv', index='False', header=True, sep='\t')
hosp_df.to_excel('hosp_data.xlsx', index='False', header=True)

In [174]:
for t in desease_types:
    hosp_df[t] = False

In [146]:
desease_types = set()
for i, t in enumerate(list(hosp_df.types)):
    if isinstance(t, str):
        t = t.split(';')
        for elem in t:
            desease_types.add(elem)

In [175]:
def fun(x):
    if isinstance(x['types'], str):
        types = x['types'].split(';')
        x[types] = 1
    return x

In [176]:
hosp_df_bin = hosp_df.apply(lambda x: fun(x), axis=1)

In [177]:
hosp_df_bin = hosp_df_bin.drop(['types'], axis = 1)

In [172]:
hosp_df_bin.to_excel('hosp_data_bin.xlsx', index='False', header=True)

In [6]:
page = requests.get(url.format(page_id))
page = page.content
page = BeautifulSoup(page, 'lxml')

In [8]:
table = page.findAll('table', attrs = {"id":"ctl00_plate_gr"})[0]

In [17]:
table_df = pd.read_html(str(table))[0]

In [102]:
table_df.columns = table_df.ix[0]
table_df = table_df.drop(0).reset_index()
del table_df['index']

In [104]:
table_df['types'] = np.nan

In [73]:
links = []
for i, tr in enumerate(table):
    if '<th' in str(tr) or isinstance(tr, bs4.element.NavigableString):
        continue
    else:
        for td in tr:
            if not (isinstance(td, bs4.element.NavigableString)):
                re_links = re.findall(r'window.location.href=\'(.+?)\'',str(td))
                if re_links:
                    links += [re_links[0]]
                    break
print(links)

['CIAcrOrgTrial.aspx?ID=65A3A2ED-8233-4697-AB5F-E3F71ED9D124', 'CIAcrOrgTrial.aspx?ID=E3CED3C1-9BFE-4BC8-A996-E1FCAB15AC49', 'CIAcrOrgTrial.aspx?ID=147850CA-5024-42DC-B1D2-12A67933A43F', 'CIAcrOrgTrial.aspx?ID=7D3C18DA-BF2A-4217-A92B-839825FCB650', 'CIAcrOrgTrial.aspx?ID=89DD7AD7-FF8D-4AE9-BA6F-CC8C40E7FA2F', 'CIAcrOrgTrial.aspx?ID=0CB6448A-EB9B-49B4-B361-7E67172C81F3', 'CIAcrOrgTrial.aspx?ID=3ACE69F9-37F8-4233-B707-8DE70A36AA9A', 'CIAcrOrgTrial.aspx?ID=A41C582A-5E38-4529-B6D3-C8F7ABAA2E99', 'CIAcrOrgTrial.aspx?ID=EE5B68D9-B18C-4576-AC14-832AC34CB66C', 'CIAcrOrgTrial.aspx?ID=D233D951-9D81-4D82-BD29-3A6F89AE5822', 'CIAcrOrgTrial.aspx?ID=EBF359A7-8F13-4B2E-9161-E75963E28CDA', 'CIAcrOrgTrial.aspx?ID=5C2DAF0D-1DFC-4A53-8F40-BD1D1BC934F1', 'CIAcrOrgTrial.aspx?ID=8A51797A-EC54-4FCB-993D-C468200DFC86', 'CIAcrOrgTrial.aspx?ID=42DEDA9B-2C2B-4D36-9621-B47D8DF56BE5', 'CIAcrOrgTrial.aspx?ID=7CF24AF6-56DC-4CEB-A4F9-B85A75A00D3C', 'CIAcrOrgTrial.aspx?ID=305BAA75-8CD2-41AE-B3F3-D55A86B39374', 'CIAcrO

In [79]:
for link in links:
    link_hosp = url_hosp + link
    hosp_page = requests.get(link_hosp)
    hosp_page = hosp_page.content
    hosp_page = BeautifulSoup(hosp_page, 'lxml')
    break

In [80]:
link_hosp

'http://grls.rosminzdrav.ru/CIAcrOrgTrial.aspx?ID=65A3A2ED-8233-4697-AB5F-E3F71ED9D124'

In [82]:
table_profiles = hosp_page.findAll('table', attrs = {"id":"ctl00_plate_grdDetailShort"})[0]

In [107]:
test = list(pd.read_html(str(table_profiles))[0][0])

In [109]:
';'.join(test)

'Профили мед. деятельности;Акушерство и гинекология;Анастезиология и реаниматология;Гематология;Дерматология;Детская онкология;Детская хирургия;Неонатология;Онкология;Ортопедия;Педиатрия;Психиатрия;ТЕРАПИЯ (ОБЩАЯ);Урология;Хирургия'

In [14]:
links = []
n = 10 # число страниц
for name, url in districts.items():
    for page in range(1, n):
        
        page_url =  url.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')
        count_pages = search_page.findAll('div', attrs = {"class":"serp-above__count"})
        print(re.findall('[0-9]+', str(count_pages)))
        break

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)

['24524']
